In [1]:
print("Test")

Test


In [ ]:
dsa_dnames.txt       # Dämonennamen
dsa_gnames.txt       # Gottheiten
dsa_names.txt        # Aventurische Personennamen
dsa_stadtnames.txt   # Städte Aventuriens


NER Trainieren

In [ ]:
import spacy
from spacy.training.example import Example
from pathlib import Path
import random

#Dateien laden
def load_list(path):
    with open(path, "r", encoding="utf8") as f:
        return [line.strip() for line in f if line.strip()]

demon_names = load_list("dsa_dnames.txt")
god_names = load_list("dsa_gnames.txt")
person_names = load_list("dsa_names.txt")
city_names = load_list("dsa_stadtnames.txt")

print(f"Loaded {len(person_names)} person names, {len(city_names)} cities.")

#Aus Listen einfache Trainingsdaten generieren
def make_training_samples(words, label):
    samples = []
    for w in words:
        sent = f"{w} erschien plötzlich in der Geschichte."
        start = sent.index(w)
        end = start + len(w)
        samples.append((sent, {"entities": [(start, end, label)]}))
    return samples

train_data = []
train_data += make_training_samples(person_names, "PER")
train_data += make_training_samples(city_names, "LOC")
train_data += make_training_samples(demon_names, "PER")   # oder custom label?
train_data += make_training_samples(god_names, "PER")     # ggf. "MISC" oder "LOC"?

random.shuffle(train_data)

print(f"Generated {len(train_data)} training examples.")

#spaCy-Modell laden und NER-Komponente vorbereiten
nlp = spacy.load("de_core_news_lg")
ner = nlp.get_pipe("ner")

# Labels hinzufügen
ner.add_label("PER")
ner.add_label("LOC")

#Training
nlp.disable_pipes([p for p in nlp.pipe_names if p != "ner"])

optimizer = nlp.initialize()

n_iters = 20

for it in range(n_iters):
    random.shuffle(train_data)
    losses = {}
    for text, annotations in train_data:
        example = Example.from_dict(nlp.make_doc(text), annotations)
        nlp.update([example], sgd=optimizer, losses=losses)
    print(f"Iteration {it+1}/{n_iters} - Losses: {losses}")

#Modell speichern
"""output_dir = Path("dsa_spacy_model")
output_dir.mkdir(exist_ok=True)
nlp.to_disk(output_dir)"""

#print(f"Model saved to {output_dir.absolute()}")

print("Training abgeschlossen.")


Loaded 6670 person names, 566 cities.
Generated 7577 training examples.
Iteration 1/20 - Losses: {'ner': np.float32(1693.5444)}
Iteration 2/20 - Losses: {'ner': np.float32(1348.3699)}
Iteration 3/20 - Losses: {'ner': np.float32(977.12573)}
Iteration 4/20 - Losses: {'ner': np.float32(760.9753)}
Iteration 5/20 - Losses: {'ner': np.float32(643.8173)}
Iteration 6/20 - Losses: {'ner': np.float32(569.2016)}
Iteration 7/20 - Losses: {'ner': np.float32(437.60535)}
Iteration 8/20 - Losses: {'ner': np.float32(389.65405)}
Iteration 9/20 - Losses: {'ner': np.float32(368.53146)}
Iteration 10/20 - Losses: {'ner': np.float32(339.902)}
Iteration 11/20 - Losses: {'ner': np.float32(319.56378)}
Iteration 12/20 - Losses: {'ner': np.float32(291.34076)}
Iteration 13/20 - Losses: {'ner': np.float32(261.15192)}
Iteration 14/20 - Losses: {'ner': np.float32(236.98045)}
Iteration 15/20 - Losses: {'ner': np.float32(277.1692)}
Iteration 16/20 - Losses: {'ner': np.float32(248.64323)}
Iteration 17/20 - Losses: {'ner

NameError: name 'output_dir' is not defined

In [12]:
test_text_real = 'Bei der Frage ob ihre Haut wirklich so dunkel sei muss Amara lachen. "Ja, die ist schon immer so." \
antwortet sie und reicht der Geweihten dann ihre Hand, damit diese sich die dunkle Haut näher \
ansehen kann. Offenbar ist sie durchaus daran gewöhnt solche Fragen zu bekommen. \
Unterdessen antwortet sie aber schon auf die übrigen Bemerkungen. \
"In Thorwal bringen wir die Nummer natürlich nicht. Aber Ihr habt natürlich recht. Es ist eine \
Provokation. Früher hatten wir das auch nicht drin. Es hat sich eigentlich eher zufällig so ergeben. \
Das Publikum reagiert eben einfach stärker auf eine kaum zu bändigende Wilde als auf eine ... \
Naja, normale Artistin die zufällig dunkle Haut hat. \
Ein mal hatten wir einen tapferen Recken im Publikum, der noch während der Vorstellung in die \
Manege gestürmt ist um mich zu befreien. Aber wir sind bei der Nummer ja hinter Gittern und ehe \
er da rein gekommen ist war ich schon hinter dem Vorhang wo wir dann alles aufklären konnten. \
Aber ich bin unhöflich. Setzt Euch doch. Darf ich Euch etwas zu trinken anbieten?"'

doc = nlp(test_text_real)
print("\nTestausgabe:\n")
for ent in doc.ents:
    print(ent.text, ent.label_)


Testausgabe:

Bei der LOC
ihre Haut wirklich so dunkel sei muss Amara lachen. "Ja, die ist schon immer so." antwortet sie und reicht der Geweihten dann ihre Hand, damit diese sich die dunkle Haut näher PER
"In Thorwal PER
wir die Nummer natürlich nicht. Aber Ihr habt natürlich recht. Es PER
eine Provokation LOC
. Früher hatten wir das auch nicht drin. Es hat sich eigentlich PER
eher zufällig so ergeben. Das Publikum reagiert eben einfach stärker auf eine kaum zu bändigende Wilde als auf eine ... Naja, normale Artistin die zufällig dunkle Haut hat. Ein mal hatten wir einen tapferen Recken im Publikum, der noch während der Vorstellung in die Manege gestürmt ist um mich zu befreien. Aber wir sind bei der Nummer ja hinter Gittern und ehe er da rein gekommen PER
Vorhang wo wir PER
bin unhöflich. Setzt Euch doch. Darf ich Euch etwas zu trinken anbieten?" PER


In [13]:
test_text_real2 = 'Siona schüttelt lächelnd den Kopf. "Ich hatte es vorhin nicht erwähnt. Ich komme aus Bjaldorn, \
falls Euch das etwas sagt." Bei der Erinnerung huscht ein kleiner Schatten über ihre Gesichtszüge, \
doch fängt sie sich schnell wieder. "Ifirn Kristallpalast ist dort, seit ein paar Jahren ist er auch \
wieder heiliger Grund und Boden." Jetzt lächelt sie schon wieder in Erinnerung an die Eisrose, \
welche Segen nach Borbarads Schergen zurück brachte.'

doc = nlp(test_text_real2)
print("\nTestausgabe:\n")
for ent in doc.ents:
    print(ent.text, ent.label_)


Testausgabe:

Siona schüttelt lächelnd den Kopf. "Ich hatte es vorhin nicht erwähnt. Ich komme aus Bjaldorn, falls Euch das etwas sagt." Bei der Erinnerung huscht ein kleiner Schatten über ihre Gesichtszüge, doch fängt sie sich schnell wieder. "Ifirn Kristallpalast ist dort, seit ein paar Jahren ist er auch wieder heiliger Grund und Boden." Jetzt lächelt sie schon wieder in Erinnerung an die Eisrose, welche Segen nach Borbarads Schergen zurück brachte. PER


In [14]:
test_text = "Borbarad reiste nach Gareth und traf auf den Dämon Belhalhar, während er zu Praios betete."

doc = nlp(test_text)
print("\nTestausgabe:\n")
for ent in doc.ents:
    print(ent.text, ent.label_)


Testausgabe:

Borbarad reiste nach Gareth und traf auf den Dämon Belhalhar, während er zu Praios betete. PER


---------------------------------- NER + EntityRuler ---------------------------------------------

In [ ]:
# -*- coding: utf-8 -*-
"""
DSA NER + EntityRuler Training (Offsets fix)
Autor: ChatGPT
"""

import spacy
from spacy.pipeline import EntityRuler
from spacy.training.example import Example
from pathlib import Path
import random


# 1. Listen laden
def load_list(path):
    return [line.strip() for line in Path(path).read_text(encoding="utf8").split("\n") if line.strip()]

daemonen = load_list("dsa_dnames.txt")
goetter  = load_list("dsa_gnames.txt")
personen = load_list("dsa_names.txt")
staedte  = load_list("dsa_stadtnames.txt")


# 2. Pre-trained Modell
nlp = spacy.load("de_core_news_lg")


# 3. EntityRuler vor NER
if "entity_ruler" not in nlp.pipe_names:
    ruler = nlp.add_pipe("entity_ruler", before="ner", config={"overwrite_ents": True})
else:
    ruler = nlp.get_pipe("entity_ruler")

patterns = []
for n in daemonen: patterns.append({"label": "DAEMON", "pattern": [{"LOWER": n.lower()}]})
for n in goetter:  patterns.append({"label": "GOTT", "pattern": [{"LOWER": n.lower()}]})
for n in personen: patterns.append({"label": "PERSON", "pattern": [{"LOWER": n.lower()}]})
for n in staedte:  patterns.append({"label": "STADT", "pattern": [{"LOWER": n.lower()}]})
ruler.add_patterns(patterns)
print(f"{len(patterns)} Muster zum EntityRuler hinzugefügt.")


# 4. NER Labels
ner = nlp.get_pipe("ner")
ner.add_label("DAEMON")
ner.add_label("GOTT")
ner.add_label("PERSON")
ner.add_label("STADT")


# 5. Automatische Trainingsdaten mit char_span Fix
import random
from spacy.training import Example

def generate_training_examples(nlp, num_examples=50):
    examples = []

    
    # Erweiterte Beispiele aus Testtext
    erweiterte_sents = [
        # Positive Beispiele
        ("Amara lacht bei der Frage.", [(0, 5, "PERSON")]),
        ("In Thorwal bringen wir die Nummer nicht.", [(3, 9, "STADT")]),
        ("Ein tapferer Recke stürmt in die Manege.", [(3, 18, "PERSON")]),
        ("Amara reicht ihre Hand.", [(0, 5, "PERSON")]),
        ("Thorwal ist eine große Stadt.", [(0, 6, "STADT")]),
        ("Ein Zuschauer stürmt auf die Bühne.", [(3, 11, "PERSON")]),
        
        # Negative Beispiele (keine Entität)
        ("Unterdessen antwortet sie schon.", []),
        ("Das Publikum reagiert stark.", []),
        ("Sie geht hinter den Vorhang.", []),
        ("Rein kommt niemand während der Vorstellung.", []),
        ("Die Nummer ist eine Provokation.", []),
        ("Er senkt den Kopf vor ihm.", []),
        ("Ich komme mit.", []),
        ("Kann ich das auch tun?", [])
    ]

    
    # Basis-Satztypen
    for _ in range(num_examples):
        p = random.choice(personen)
        s = random.choice(staedte)
        g = random.choice(goetter)
        d = random.choice(daemonen)

        s1 = f"{p} reiste nach {s}."
        s2 = f"{g} sandte einen Dämon namens {d}."
        s3 = f"{p} traf {g} in {s}."
        s4 = f"Ein Dämon namens {d} erschien vor {p}."
        s5 = f"{g} erschien in {s} und sprach zu {p}."
        s6 = f"{p} kämpfte gegen den Dämon {d} in {s}."
        s7 = f"{g} und {d} waren die Ursache für Chaos in {s}."
        s8 = f"{p} erhielt einen Auftrag von {g}, einen Dämon namens {d} zu besiegen."
        s9 = f"{d} wurde von {p} in {s} gebannt."
        s10 = f"{p}, {g} und {d} trafen sich in {s} zu einem Ritual."

        sents_and_ents = [
            (s1, [(p,"PERSON"), (s,"STADT")]),
            (s2, [(g,"GOTT"), (d,"DAEMON")]),
            (s3, [(p,"PERSON"), (g,"GOTT"), (s,"STADT")]),
            (s4, [(d,"DAEMON"), (p,"PERSON")]),
            (s5, [(g,"GOTT"), (s,"STADT"), (p,"PERSON")]),
            (s6, [(p,"PERSON"), (d,"DAEMON"), (s,"STADT")]),
            (s7, [(g,"GOTT"), (d,"DAEMON"), (s,"STADT")]),
            (s8, [(p,"PERSON"), (g,"GOTT"), (d,"DAEMON")]),
            (s9, [(d,"DAEMON"), (p,"PERSON"), (s,"STADT")]),
            (s10, [(p,"PERSON"), (g,"GOTT"), (d,"DAEMON"), (s,"STADT")])
        ]

        # Alle Basis-Sätze hinzufügen
        for sent, ents in sents_and_ents:
            doc = nlp.make_doc(sent)
            spans = []
            for text_val, label in ents:
                span = doc.char_span(
                    sent.index(text_val),
                    sent.index(text_val)+len(text_val),
                    label=label,
                    alignment_mode="contract"
                )
                if span:
                    spans.append(span)
            doc.ents = spans
            ex = Example.from_dict(doc, {"entities": [(e.start_char, e.end_char, e.label_) for e in spans]})
            examples.append(ex)

    # Erweiterte Sätze hinzufügen 
    for sent, ents in erweiterte_sents:
        doc = nlp.make_doc(sent)
        spans = []
        for start, end, label in ents:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span:
                spans.append(span)
        doc.ents = spans
        ex = Example.from_dict(doc, {"entities": [(e.start_char, e.end_char, e.label_) for e in spans]})
        examples.append(ex)

    return examples



# Training-Daten generieren
TRAIN_DATA = generate_training_examples(nlp, num_examples=50)  # 500 Sätze

# 6. Training
optimizer = nlp.resume_training()

for i in range(60):
    losses = {}
    nlp.update(TRAIN_DATA, sgd=optimizer, losses=losses)
    print(f"Iteration {i+1} – Losses: {losses}")


# 7. Modell speichern
OUTPUT_MODEL = "dsa_ner_model_clean"
nlp.to_disk(OUTPUT_MODEL)
print(f"Modell gespeichert unter: {OUTPUT_MODEL}")


/home/timc/Workspace/nlp-rpg-narratives/nlp-rpg-venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


7577 Muster zum EntityRuler hinzugefügt.
Iteration 1 – Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'lemmatizer': 0.0, 'ner': np.float32(2724.7185)}
Iteration 2 – Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'lemmatizer': 0.0, 'ner': np.float32(2413.5)}
Iteration 3 – Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'lemmatizer': 0.0, 'ner': np.float32(2261.4216)}
Iteration 4 – Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'lemmatizer': 0.0, 'ner': np.float32(2141.977)}
Iteration 5 – Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'lemmatizer': 0.0, 'ner': np.float32(2030.1023)}
Iteration 6 – Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'lemmatizer': 0.0, 'ner': np.float32(1904.1267)}
Iteration 7 – Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'lemmatizer': 0.0, 'ner': np.flo

In [ ]:
test_text_real = 'Bei der Frage ob ihre Haut wirklich so dunkel sei muss Amara lachen. "Ja, die ist schon immer so." \
antwortet sie und reicht der Geweihten dann ihre Hand, damit diese sich die dunkle Haut näher \
ansehen kann. Offenbar ist sie durchaus daran gewöhnt solche Fragen zu bekommen. \
Unterdessen antwortet sie aber schon auf die übrigen Bemerkungen. \
"In Thorwal bringen wir die Nummer natürlich nicht. Aber Ihr habt natürlich recht. Es ist eine \
Provokation. Früher hatten wir das auch nicht drin. Es hat sich eigentlich eher zufällig so ergeben. \
Das Publikum reagiert eben einfach stärker auf eine kaum zu bändigende Wilde als auf eine ... \
Naja, normale Artistin die zufällig dunkle Haut hat. \
Ein mal hatten wir einen tapferen Recken im Publikum, der noch während der Vorstellung in die \
Manege gestürmt ist um mich zu befreien. Aber wir sind bei der Nummer ja hinter Gittern und ehe \
er da rein gekommen ist war ich schon hinter dem Vorhang wo wir dann alles aufklären konnten. \
Aber ich bin unhöflich. Setzt Euch doch. Darf ich Euch etwas zu trinken anbieten?"'

doc = nlp(test_text_real)
print("\nTestausgabe:\n")
for ent in doc.ents:
    print(ent.text, ent.label_)


Testausgabe:

Amara PERSON
Ja GOTT
Thorwal STADT
Wilde PERSON
Naja GOTT
Euch GOTT
Darf GOTT


In [4]:
test_text_real2 = 'Siona schüttelt lächelnd den Kopf. "Ich hatte es vorhin nicht erwähnt. Ich komme aus Bjaldorn, \
falls Euch das etwas sagt." Bei der Erinnerung huscht ein kleiner Schatten über ihre Gesichtszüge, \
doch fängt sie sich schnell wieder. "Ifirn Kristallpalast ist dort, seit ein paar Jahren ist er auch \
wieder heiliger Grund und Boden." Jetzt lächelt sie schon wieder in Erinnerung an die Eisrose, \
welche Segen nach Borbarads Schergen zurück brachte.'

doc = nlp(test_text_real2)
print("\nTestausgabe:\n")
for ent in doc.ents:
    print(ent.text, ent.label_)


Testausgabe:

Siona PERSON
Bjaldorn PERSON
Ifirn PERSON
Kristallpalast PERSON
Eisrose PERSON
Borbarads STADT


In [5]:
test_text = "Borbarad reiste nach Gareth und traf auf den Dämon Belhalhar, während er zu Praios betete."

doc = nlp(test_text)
print("\nTestausgabe:\n")
for ent in doc.ents:
    print(ent.text, ent.label_)


Testausgabe:

Borbarad PERSON
Gareth PERSON
Dämon PERSON
Belhalhar DAEMON
Praios PERSON


In [6]:
doc = nlp("In der Kapelle der Travia sprach die Geweihte mit Praiodane aus Nordhag.")

for ent in doc.ents:
    print(ent.text, ent.label_)


Travia PERSON
Praiodane PERSON
Nordhag STADT


In [10]:
nlp = spacy.load("de_core_news_lg")
doc = nlp(test_text)

for ent in doc.ents:
    print(ent.text, ent.label_)


Borbarad PER
Gareth LOC
Belhalhar PER
Praios PER


Speichern:

In [ ]:
nlp.to_disk("dsa_nlp_hybrid")


Laden: 

In [ ]:
nlp = spacy.load("dsa_nlp_hybrid")
